<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
shares = ['ALRS','GAZP', 'SIBN','PLZL','GMKN','MAGN']

In [3]:
import requests
import sqlite3

from time import sleep
from datetime import datetime
from pytz import timezone

In [18]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'accept': '*/*',
    'content-type': 'application/x-www-form-urlencoded',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'
}
link = 'https://www.tinkoff.ru/api/invest-gw/social/v1/post/instrument/{}?limit=50&appName=invest&platform=web&cursor={}'


conn = sqlite3.connect('pulse_lesson.db')
cursor = conn.cursor()

### Переделать структуру БД чтобы можно было пользоваться !!!!!

def _init_database():
    with open('C:/Users/Iljam/create_table.sql', 'r') as f:
        sql = f.read()
    cursor.execute(sql)
    conn.commit()


def insert(table_name: str, column_data: dict) -> None:
    columns = ', '.join(column_data.keys())
    values = [tuple(column_data.values())]
    placeholders = ', '.join('?' * len(column_data.keys()))
    cursor.executemany(
        f'insert into {table_name}({columns}) values({placeholders})', values
    )
    conn.commit()


def get_instruments(lst: list):
    data = []
    for item in lst:
        line = f"{item.get('briefName')} !! {item.get('ticker')} !! {item.get('price')} !! {item.get('lastPrice')}"
        data.append(line)
    return data


def convert_inserted(line):
    utc_time = datetime.strptime(line, '%Y-%m-%dT%H:%M:%S.%f%z')\
        .replace(tzinfo=timezone('utc')).strftime('%Y-%m-%d %H:%M:%S')
    return utc_time


def get_cursor(url: str, ticker: str, cursor_number: str) -> str:
    link = url.format(ticker, cursor_number)
    session = requests.Session()
    data = session.get(link, headers=headers, stream=True)
    cursor_number = data.json().get('payload').get('nextCursor')
    return cursor_number


def get_data_from_api(url: str, ticker: str, cursor_number: str) -> None:
    link = url.format(ticker, cursor_number)
    session = requests.Session()
    data = session.get(link, headers=headers, stream=True)
    raw_data = data.json().get('payload').get('items')
    

    for i in raw_data:
        my_data = {
            'ticker': ticker,
            'id': i.get('id')[:7],
            'inserted': convert_inserted(i.get('inserted')),
            'instruments': ', '.join(get_instruments(i["content"].get('instruments'))),
            'likesCount': i['reactions'].get('totalCount'),
            'nickname': i.get('nickname'),
            'commentsCount': i.get('commentsCount'),
            'parse_date': datetime.today().strftime('%Y-%m-%d %H:%M:%S'),
            'text': i.get('content').get('text')
        }
        insert('dt_pulse_GAZP', my_data)


def get_data_from_ticker(ticker: str):
    max_cursor = '999999999999999'
    for _ in tqdm(range(1, 100)):
        get_data_from_api(link, ticker, max_cursor)
        sleep(1.8)
        max_cursor = get_cursor(link, ticker, max_cursor)
        sleep(1)


def main():
    get_data_from_ticker("F")
    
#if __name__ == '__main__':
    #main()  

In [31]:
from sqlite3 import connect
conn = connect('pulse_lesson.db')
db_name = "dt_pulse_GAZP"
df = pd.read_sql(sql= f"SELECT * FROM {db_name}", con=conn)

In [32]:
df = df[df['ticker'] == "GAZP"]

In [33]:
df

,src_id,ticker,id,inserted,instruments,likesCount,nickname,commentsCount,parse_date,text
0,None,GAZP,133dd47,2023-01-12 15:32:38,"Yandex !! YNDX !! 1894.4 !! 1894.6, Сбер Банк ...",0,Kadey_investor,0,2023-01-12 15:34:25,Снова пополнение инвестиционного счета 😊 в это...
1,None,GAZP,f9e8fde,2023-01-12 15:24:37,Газпром !! GAZP !! 165.03 !! 164.7,0,Rauf_,1,2023-01-12 15:34:25,"{$GAZP} Всем привет ,подскажите кто знает по И..."
2,None,GAZP,dbbed1e,2023-01-12 15:13:39,Газпром !! GAZP !! 164.98 !! 164.7,2,IvanVer,1,2023-01-12 15:34:25,{$GAZP}
3,None,GAZP,3f5e13d,2023-01-12 14:57:08,Газпром !! GAZP !! 165.02 !! 164.7,0,Slim78,7,2023-01-12 15:34:25,{$GAZP} Когда дивидендов ждать?
4,None,GAZP,d56143e,2023-01-12 14:56:13,Газпром !! GAZP !! 165.01 !! 164.7,5,INVESTOMATIKA,3,2023-01-12 15:34:25,📈 В предыдущий торговый день акции компании Га...
...,...,...,...,...,...,...,...,...,...,...
90930,None,GAZP,c0b9eb0,2023-01-20 13:22:19,Газпром !! GAZP !! 158.96 !! 159.97,0,simivann,3,2023-01-23 22:06:59,"{$GAZP} вроде пакистанский поток, выход на без..."
90931,None,GAZP,5965cf7,2023-01-20 13:20:44,Газпром !! GAZP !! 159.05 !! 159.97,8,A0f,2,2023-01-23 22:06:59,"{$GAZP} похоже банкрот, одна надежда, что его ..."
90932,None,GAZP,1fd2a11,2023-01-20 13:18:51,Газпром !! GAZP !! 159.06 !! 159.97,0,chapokedy,0,2023-01-23 22:06:59,{$GAZP} Баюшки баю
90933,None,GAZP,119ad6b,2023-01-20 13:18:50,Газпром !! GAZP !! 159.06 !! 159.97,3,qlex,0,2023-01-23 22:06:59,"{$GAZP} боковик и ныне там, всем хорошего наст..."


In [ ]:
# Emotional analisis
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rurewiews')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rurewiews', return_dict=True)


def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [40]:
tonality_list = []
for i in tqdm(range(len(df)-90000)):
    tonality_list.append(predict(df.loc[i, 'text']))

  0%|          | 0/935 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df['tonality_type'] = tonality_list

In [ ]:
df.to_sql("df_GAZP", con = conn)

In [ ]:
df['inserted'] = pd.to_datetime(df['inserted'])
date_list = []
for date in tqdm(range(len(df))):
    date_list.append(df.loc[date, 'inserted'].date())

In [ ]:
df['date'] = date_list

In [ ]:
month_list = []
day_list = []
for element in df['date']:
    day_list.append(element.day)
    month_list.append(element.month)
    
df['month'] = month_list
df['day'] = day_list

In [ ]:
year_list = []
for element in df['date']:
    year_list.append(element.year)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()
    
def get_sentiment(text, return_type='label'):
    """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([-1, 0, 1])
    return proba

In [ ]:
label_list = []
score_list = []

for i in tqdm(range(len(df))):
    label_list.append(get_sentiment(df.loc[i, 'text'], 'label'))
    score_list.append(get_sentiment(df.loc[i, 'text'], 'score'))

In [ ]:
df['tonality_label'] = label_list
df['tonality_score'] = score_list

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()
    
def text2toxicity(text, aggregate=True):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

In [ ]:
toxicity_list = []

for i in tqdm(range(len(df))):
    toxicity_list.append(text2toxicity(df.loc[i, 'text'], True))

In [ ]:
df['toxic_value'] = toxicity_list

In [ ]:
df = df.drop(['src_id','id','instruments','nickname','parse_date','day'], axis = 1)
df.head()

In [ ]:
df.to_sql('df_GAZP_2', con = conn)

In [ ]:
import torch
from transformers import BertForSequenceClassification, AutoTokenizer

LABELS = ['neutral', 'happiness', 'sadness', 'enthusiasm', 'fear', 'anger', 'disgust']
tokenizer = AutoTokenizer.from_pretrained('Aniemore/rubert-tiny2-russian-emotion-detection')
model = BertForSequenceClassification.from_pretrained('Aniemore/rubert-tiny2-russian-emotion-detection')


def predict_emotion(text: str) -> str:
    """
        We take the input text, tokenize it, pass it through the model, and then return the predicted label
        :param text: The text to be classified
        :type text: str
        :return: The predicted emotion
    """
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
        
    return LABELS[predicted[0]]


def predict_emotions(text: str) -> list:
    """
        It takes a string of text, tokenizes it, feeds it to the model, and returns a dictionary of emotions and their
        probabilities
        :param text: The text you want to classify
        :type text: str
        :return: A dictionary of emotions and their probabilities.
    """
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    emotions_list = {}
    for i in range(len(predicted.numpy()[0].tolist())):
        emotions_list[LABELS[i]] = predicted.numpy()[0].tolist()[i]
    return emotions_list

In [ ]:
emotion_list = []

for i in tqdm(range(len(df))):
    emotion_list.append(predict_emotion(df.loc[i, 'text']))

In [ ]:
df['emotions'] = emotion_list

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
month_year = []
for date in df['date']:
    month_year.append(date.strftime("%Y-%m"))

In [ ]:
df['month_year'] = month_year

In [ ]:
pivot_df = df.pivot_table(index = 'month_year', values = 'tonality_score', aggfunc='mean')
pivot_df

In [ ]:
fig = plt.figure(figsize = (16,8))
sns.barplot(data = pivot_df, x=pivot_df.index, y= 'tonality_score')

In [ ]:
another_agg = df['tonality_label'].value_counts()
fig = plt.figure(figsize = (8,8))
plt.pie(x = another_agg)

In [ ]:
df.to_sql("df_pulse_GAZP_6", con = conn)